use newpycaret env

In [1]:
# packages
import snowflake.connector
import pandas as pd
import os
import numpy as np
from autots import AutoTS
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime

2. Load Data
_______________________________________________________________

### To Do: copy and paste in to a new chunk, enter credentials and run to save in environment. Then delete chunk
%env snowflakeuser=<your_snowflake_username> <br>
%env snowflakepass=<your_snowflake_password>

In [27]:
# Query Snowflake

# Snowflake connection parameters
connection_params = {
    "user": os.environ['snowflakeuser'],
    "password": os.environ['snowflakepass'],
    "account": "zib52348.us-east-1",
    "role": "ACCOUNTADMIN",
    "warehouse": "REPORTING",
    "database": "ANALYTICS",
    "schema": "FORECASTING",
}

# Establish a connection to Snowflake
conn = snowflake.connector.connect(**connection_params)

with open('net_sales_query.sql', 'r') as query:
    # connection == the connection to your database, in your case prob_db
    df_in = pd.read_sql_query(query.read(),conn)

# Close the connection
conn.close()

C:\Users\crudek\AppData\Local\Temp\ipykernel_4872\3034601435.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_in = pd.read_sql_query(query.read(),conn)


In [28]:
### 1. Data prep ###

df = df_in

# convert month field to date
df["MONTH"] = pd.to_datetime(df["MONTH"])

# Some random months will have data that we want to remove (* Want to test without July though)
df = df[["DEP_ENT", "MONTH", "NET_SALES"]] # select fields of interest
df = df.sort_values(['DEP_ENT', 'MONTH']) # reorder dataframe


### 2. Spot checks ###

# check data types
print(df.dtypes)
print("-----")

# check the unique time_series. 14 different department-entity combinations
print(df['DEP_ENT'].nunique())
print("-----")

# check how many months for each dep-ent. 3 dep-ent do not have all months of data
print(df.groupby(['DEP_ENT']).size())
print("-----")

# check for nulls. No nulls
print(df.isnull().sum())

DEP_ENT              object
MONTH        datetime64[ns]
NET_SALES           float64
dtype: object
-----
14
-----
DEP_ENT
160_155    92
170_155    92
200_155    96
200_310    94
210_155    92
210_165    92
210_310    92
220_155    92
220_310    86
240_155    92
250_155    92
250_165    58
250_310    92
260_155    55
dtype: int64
-----
DEP_ENT      0
MONTH        0
NET_SALES    0
dtype: int64


In [29]:
### Additional data prep ###

# create series
df_s = df.set_index(['DEP_ENT','MONTH'])['NET_SALES']
# convert back to dataframe
df_d = df_s.to_frame()
# impute
mean_imputer = SimpleImputer(strategy='mean')
df_d['NET_SALES'] = mean_imputer.fit_transform(df_d['NET_SALES'].values.reshape(-1,1))
#reset index
df_d.reset_index(inplace=True)

In [30]:
### Regressor data prep

# create copy of df_d
df_d_regress = df_d.copy(deep=True)

## Prepare regressors

# Define the prime COVID period
cov_start_date = pd.Timestamp(2020,1,1)
cov_end_date = pd.Timestamp(2021,12,1)

# Create the binary dummy variable
df_d_regress['COVID'] = df_d_regress['MONTH'].apply(lambda date: '1' if cov_start_date <= date <= cov_end_date else '0')

# Amazon prime day shipments

# Define the dictionary
amzn_ship_dict = {
    'MONTH': [pd.Timestamp(2016,7,1), pd.Timestamp(2017,5,1), pd.Timestamp(2018,6,1), pd.Timestamp(2019,6,1), pd.Timestamp(2020,6,1),
pd.Timestamp(2020,10,1), pd.Timestamp(2021,5,1), pd.Timestamp(2022,5,1), pd.Timestamp(2023,5,1), pd.Timestamp(2023,6,1), pd.Timestamp(2023,7,1)],
    'AMZN': ['1', '1', '1','1', '1', '1','1', '1', '1','1', '1']
}

# Create a DataFrame from the dictionary
amzn_ship_df = pd.DataFrame(amzn_ship_dict)
# add Amazon department
amzn_ship_df['DEP_ENT'] = '250_155'

# combine in to single table
df_d_regress1 = df_d_regress.merge(amzn_ship_df, on=['DEP_ENT', 'MONTH'], how = 'left')

df_d_regress1['AMZN'] = df_d_regress1['AMZN'].fillna(0)

____________________________________________________________________________________________________________________________________________________________________________

In [34]:
### Time Series Loop w/ Regressors ###

# Create empty dataframes
all_predictions = pd.DataFrame()

# list of each dep-ent
all_ts_ts = df_d['DEP_ENT'].unique()

for i in tqdm(all_ts_ts):

    df_subset = df_d_regress1[df_d_regress1['DEP_ENT'] == i]

    df_subset_trim = df_subset[df_subset['MONTH'] <= pd.Timestamp(2022, 12, 1)]
    df_subset_trim = df_subset_trim[['MONTH', 'NET_SALES']]
    # split the train and original df

    # train
    df_subset_train = df_subset[df_subset['MONTH'] <= pd.Timestamp(2022, 12, 1)]
    # select fields of interest
    df_subset_train = df_subset_train[["MONTH", "COVID", "AMZN"]]

    # for future forecast
    df_subset_fcast = df_subset[df_subset['MONTH'] > pd.Timestamp(2022, 12, 1)]
    # select fields of interest
    df_subset_fcast = df_subset_fcast[["MONTH", "COVID", "AMZN"]]

# model
model = AutoTS(
    forecast_length=12,
    frequency='infer',
    # prediction_interval=0.95,
    # ensemble='simple',
    # models_mode='deep',
    # model_list = 'univariate', # or could do a list like ['ARIMA','ETS']
    max_generations=3,
    num_validations=3,
    no_negatives=True,
    n_jobs='auto')

model = model.fit(
    df_subset_trim,
    future_regressor=df_subset_train,
    date_col='MONTH',
    value_col='NET_SALES',
)

#print(model)

# create prediction
prediction = model.predict(future_regressor=df_subset_fcast)

# temp fcast dataframe
temp_fcasts = prediction.forecast

# rename
temp_fcasts.rename(columns={'index': 'MONTH'}, inplace=True)

temp_fcasts['DEP_ENT'] = i  # add dep

# append to master dataframe
all_predictions = all_predictions.append(temp_fcasts)

100%|██████████| 14/14 [00:00<00:00, 468.05it/s]


Using 6 cpus for n_jobs.
Data frequency is: MS, used frequency is: MS
Too many training validations for length of data provided, decreasing num_validations to 2
Model Number: 1 with model ARIMA in generation 0 of 3
Model Number: 2 with model AverageValueNaive in generation 0 of 3
Model Number: 3 with model AverageValueNaive in generation 0 of 3
Model Number: 4 with model AverageValueNaive in generation 0 of 3
Model Number: 5 with model DatepartRegression in generation 0 of 3
Model Number: 6 with model DatepartRegression in generation 0 of 3
Model Number: 7 with model DatepartRegression in generation 0 of 3
Model Number: 8 with model DatepartRegression in generation 0 of 3
Template Eval Error: ImportError('Tensorflow not available, install with pip install tensorflow.') in model 8 in generation 0: DatepartRegression
Model Number: 9 with model ETS in generation 0 of 3


c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model Number: 10 with model ETS in generation 0 of 3
Model Number: 11 with model GLM in generation 0 of 3
Model Number: 12 with model GLM in generation 0 of 3
Model Number: 13 with model GLS in generation 0 of 3
Model Number: 14 with model GLS in generation 0 of 3
Model Number: 15 with model GluonTS in generation 0 of 3
Template Eval Error: ImportError('GluonTS installation is incompatible with AutoTS. The numpy version is sometimes the issue, try 1.23.1 {as of 06-2023}') in model 15 in generation 0: GluonTS
Model Number: 16 with model GluonTS in generation 0 of 3
Template Eval Error: ImportError('GluonTS installation is incompatible with AutoTS. The numpy version is sometimes the issue, try 1.23.1 {as of 06-2023}') in model 16 in generation 0: GluonTS
Model Number: 17 with model GluonTS in generation 0 of 3
Template Eval Error: ImportError('GluonTS installation is incompatible with AutoTS. The numpy version is sometimes the issue, try 1.23.1 {as of 06-2023}') in model 17 in generation

14:47:21 - cmdstanpy - INFO - Chain [1] start processing
14:47:21 - cmdstanpy - INFO - Chain [1] done processing


...

c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:307: DomainWarning: The InversePower link function does not respect the domain of the Gamma family.
  warnings.warn((f"The {type(family.link).__name__} link function "


70 - NVAR with avg smape 56.19: 
Model Number: 71 of 84 with model GLM for Validation 2
📈 71 - GLM with avg smape 51.22: 
Model Number: 72 of 84 with model MultivariateMotif for Validation 2
Template Eval Error: ValueError('window shape cannot be larger than input array shape') in model 72 in generation 0: MultivariateMotif
Model Number: 73 of 84 with model MultivariateRegression for Validation 2
📈 73 - MultivariateRegression with avg smape 51.18: 
Model Number: 74 of 84 with model AverageValueNaive for Validation 2
74 - AverageValueNaive with avg smape 56.19: 
Model Number: 75 of 84 with model Theta for Validation 2
Template Eval Error: ValueError('x must have 2 complete cycles requires 24 observations. x only has 11 observation(s)') in model 75 in generation 0: Theta
Model Number: 76 of 84 with model Theta for Validation 2
Template Eval Error: ValueError('x must have 2 complete cycles requires 24 observations. x only has 11 observation(s)') in model 76 in generation 0: Theta
Model Nu

c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1257: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1257: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
c:\Users\crudek\AppData\Local\anaconda3\envs\newpycaret\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1257: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
14:49:09 - cmdstanpy - INFO - Chain [1] start processing
14:49:09 - cmdstanpy - INFO - Chain [1] done processing


📈 82 - FBProphet with avg smape 50.71: 
Model Number: 83 of 84 with model AverageValueNaive for Validation 2
83 - AverageValueNaive with avg smape 56.19: 
Model Number: 84 of 84 with model AverageValueNaive for Validation 2
84 - AverageValueNaive with avg smape 59.23: 
TotalRuntime missing in 5!
Validation Round: 1
TotalRuntime missing in 0!
Validation Round: 2
TotalRuntime missing in 0!


C:\Users\crudek\AppData\Local\Temp\ipykernel_4872\1234459847.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_predictions = all_predictions.append(temp_fcasts)


In [16]:
# rename index col
all_predictions.rename(columns={'index': 'MONTH', 'NET_SALES': 'PRED'}, inplace=True)

#initial sales pull
sales = df_in
# select cols
sales = sales[["DEP_ENT", "MONTH", "NET_SALES"]]
# only include sales data for the full months we have
first_of_month = datetime.today().replace(day=1).date()
sales = sales[sales['MONTH'] < pd.to_datetime(first_of_month)]

# combine prediction data and original sales data
merged = pd.merge(all_predictions, sales, how = 'left', on =['DEP_ENT', 'MONTH'])

In [18]:
### Budget ###

# Establish a connection to Snowflake
conn = snowflake.connector.connect(**connection_params)

with open('budget_query.sql', 'r') as query:
    # connection == the connection to your database, in your case prob_db
    budg = pd.read_sql_query(query.read(),conn)

# Close the connection
conn.close()

# convert month to datetime
budg["MONTH"] = pd.to_datetime(budg["MONTH"])
# select cols
budg = budg[["MONTH", "BUDGET_AMOUNT", "DEP_ENT"]]

WARNING - (py.warnings._showwarnmsg) - C:\Users\crudek\AppData\Local\Temp\ipykernel_12204\3534014428.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  budg = pd.read_sql_query(query.read(),conn)



In [19]:
# combine prediction/sales data with budget data
merged2 = pd.merge(merged, budg, how = 'left', on =['DEP_ENT', 'MONTH'])

In [20]:
# write csv
merged2.to_csv('auto_ts_results.csv')